In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import requests
import urllib.request
import time

authors = []
dates = []
statements = []
content = []
sources = []
targets = []

In [ ]:
def scrape_website(page_number):
  pgno = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+ pgno +'&ruling=barely-true'
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text,'html.parser')
  #Get location
  statement_footer = soup.find_all('footer',attrs ={'class':'m-statement__footer'})
  statement_quote = soup.find_all('div',attrs={'class':'m-statement__quote'})
  statement_meta = soup.find_all('div',attrs={'class':'m-statement__meta'})
  target = soup.find_all('div',attrs={'class':'m-statement__meter'})

  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month + ' '+ day + ' '+year
    dates.append(date)
    authors.append(full_name)

  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    statements.append(statement_text)
    atag=i.find_all('a')[0]
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    URL2 = 'https://www.politifact.com' +  atag['href']
    webpage = requests.get(URL2)
    soup2 = BeautifulSoup(webpage.text)
    statement_content = soup2.find_all('article',attrs={'class':'m-textblock'})
    content.append(''.join(statement_content[0].text.strip().splitlines()))


  for i in statement_quote:
   atag=i.find_all('a')[0]   

  for i in statement_meta:
    link3 = i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  for i in target:
      link4 = i.find('div',attrs={'class':'c-image'}).find('img').get('alt')
      targets.append(link4)


In [ ]:
n = 30
for i in range(1,n):
  scrape_website(i)

In [ ]:
data = pd.DataFrame(columns = ['author','statement','article','source','date','target'])
data['author'] = pd.Series(authors)
data['statement']= pd.Series(statements)
data['source']= pd.Series(sources)
data['article']=pd.Series(content)
data['date']= pd.Series(dates)
data['target']= pd.Series(targets)

data

,author,statement,article,source,date,target
0,Tom Kertscher,"""In the month since Republicans took the House...",A post shared on Facebook left out important d...,Facebook posts,"February 10, 2023",barely-true
1,Loreben Tuquero,"The 87,000 new IRS agents who were supposed to...",If you work in the service industry and rely o...,Mike Palicz,"February 10, 2023",barely-true
2,Tom Kertscher,Wind farm construction is a possible cause of ...,Is a surge in whale deaths on the Atlantic Coa...,Instagram posts,"February 8, 2023",barely-true
3,Yacob Reyes,"""I got a list of 20 jurisdictions that defunde...","Video footage of Memphis, Tennessee, police of...",Jim Jordan,"February 1, 2023",barely-true
4,Ciara O'Rourke,The United Kingdom banned COVID-19 booster vac...,A screenshot of a blog post headline populatin...,Viral image,"January 31, 2023",barely-true
...,...,...,...,...,...,...
865,Amy Sherman,"""Terrorists who are currently being held at Gu...",A Florida Republican claims a measure passed i...,Greg Steube,"July 19, 2019",barely-true
866,Samantha Putterman,"""No one mentions that illegals are free to lea...",Border patrol facilities holding immigrants ar...,Facebook posts,"July 17, 2019",barely-true
867,Louis Jacobson,"""A black person is four times as likely as a w...",A July 4 event featuring Democratic presidenti...,Pete Buttigieg,"July 10, 2019",barely-true
868,Daniel Funke,"Migrants at the U.S. border are ""able to pay c...",We’ve fact-checked a lot of misinformation abo...,Facebook posts,"July 10, 2019",barely-true


In [ ]:
data.to_csv('New_False1.csv')